In [2]:
#2021.07.05. MON
#Hankyeong

#00. 패키지 호출
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import warnings 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.utils import to_categorical

#00-1. 시각화 옵션 설정
%matplotlib inline

#00-2. warning message ignore
warnings.filterwarnings(action='ignore')

#00-3. 씨드넘버 설정
np.random.seed(2021)
tf.random.set_seed(2021)


In [3]:
#05. 텍스트 데이터 전처리하기. 
#(1) 텍스트 데이터 생성하기. 
text = '''
    경마장에 있는 말이 뛰고 있다. \n 
    그의 말이 법이다. \n
    가는 말이 고와야 오는 말이 곱다. \n
'''

In [4]:
#(2) Tonkenizer 객체 지정하기. 
t = Tokenizer()

#(3) text를 Token으로 분리하기(토큰화하기). 
t.fit_on_texts([text])

#(4) Token의 인덱스값 파악하기. 
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [5]:
#PLUS. Token들의 빈도 확인하기.
t.word_counts 

OrderedDict([('경마장에', 1),
             ('있는', 1),
             ('말이', 4),
             ('뛰고', 1),
             ('있다', 1),
             ('그의', 1),
             ('법이다', 1),
             ('가는', 1),
             ('고와야', 1),
             ('오는', 1),
             ('곱다', 1)])

In [6]:
#(5) 단어 집합 설정하기. 
voca_size = len(t.word_index) +1 
voca_size

12

In [7]:
#WHY? keras의 Tokenizer 객체는 토큰화 시 인덱스 번호가 1부터 시작함.
#     허나 원핫인코딩 처리시 배열의 인덱스가 0부터 시작하기 때문에, 
#     크기를 동일하게 하기 위해 실제 단어 집합의 크기보다 +1로 설정해야함. 

In [8]:
#PLUS. sequence를 index 번호로 변환(벡터화)하기. 
t.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [9]:
#(6) 문장들을 index 번호로 변환하기. 
sequences = [] 
for line in text.split('\n') : 
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)) :
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
#(7) 패딩 처리 전 sequence의 최대 원소 수 파악하기.
max_len = max([len(sequence) for sequence in sequences])
max_len

6

In [11]:
#(8) 패딩 처리하기. 
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [12]:
#MEMO. pad_sequences() 메서드의 padding 파라미터에서 'pre'을 주면 앞으로(좌측으로) 0을 채우는 패딩 작용함. 

In [13]:
#(9) feature, target 데이터셋으로 분할하기. 
X = sequences[:,:-1]
y = sequences[:,-1]

In [14]:
#(10) target 변수 원핫인코딩처리하기.  
Y = to_categorical(y, voca_size)                              
Y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [15]:
#(11) feature, target 변수의 차원 확인하기. 
X.shape, Y.shape

((11, 5), (11, 12))

In [16]:
#06. Simple RNN을 이용해 텍스트 (미리보기) 생성하기. 
#(1) 모델 정의하기. 
model = Sequential([
    Embedding(voca_size, 4, input_length=(max_len-1)),
    SimpleRNN(32),
    Dense(voca_size, activation='softmax')
])

#(2) 모델의 요약 정보 확인하기. 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 4)              48        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1184      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [17]:
#(3) 모델의 compile 설정하기. 
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

#(4) 모델 학습하기. 
model_fit = model.fit(X,Y, epochs=200, verbose=0)

#(5) 훈련 최종 accuracy 확인하기. 
model_fit.history['accuracy'][-1]

0.9090909361839294

In [18]:
#07. 모델 검증하기. 
#(1) 검증용 함수 정의하기. (by github.com/ckiekim)
def sentence_generation(model, t, max_len, current_word, n) :                   # 모델, 토크나이저, 최대글자수, 현재 단어, 반복할 횟수
    init_word = current_word                                                    # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n) :                                                         # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]                       # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')     # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)                     # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.   
        for word, index in t.word_index.items() : 
            if index == result:                                                 # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면, 예측단어이므로 break
                break
        current_word = current_word + ' '  + word                               # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word                                        # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

#(2) 검증하기.
sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 말이 말이 뛰고'

In [19]:
sentence_generation(model, t, max_len, '그의', 2)

'그의 말이 법이다'

In [20]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'